In [1]:
import os
from google.colab import drive

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import nltk


import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam


from tensorflow.keras.layers import TextVectorization, Dense, MaxPooling1D, Conv1D, LSTM, MultiHeadAttention, Flatten, Layer, LayerNormalization, Dropout, Embedding, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Setting up TPU

In [2]:
os.environ['TPU_NAME']

'grpc://10.69.223.74:8470'

In [3]:
tpu_address = os.environ['TPU_NAME']
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.experimental.TPUStrategy(tpu)

print('Running on TPU: ', tpu.cluster_spec().as_dict()['worker'])
print('Number of accelerators: ', strategy.num_replicas_in_sync)


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.69.223.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.69.223.74:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU:  ['10.69.223.74:8470']
Number of accelerators:  8


### Dataset

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
datasets_path = "/content/drive/My Drive/Machine Learning Datasets/Hindi-English"

In [24]:
Hindi_embedding_matrix = np.load(datasets_path + '/Hindi_embedding_matrix.npy')
English_embedding_matrix = np.load(datasets_path + '/English_embedding_matrix.npy')

Hindi_vectorized_text = np.load(datasets_path + '/Hindi_vectorized.npy')
English_vectorized_text = np.load(datasets_path + '/English_vectorized.npy')

In [25]:
print(
    f"""
    Hindi Embeddings Shape =  {Hindi_embedding_matrix.shape}
    English Embeddings shape = {English_embedding_matrix.shape}

    Hindi_vectors_shape = {Hindi_vectorized_text.shape}
    English_vectors_shape = {English_vectorized_text.shape}
    """
)


    Hindi Embeddings Shape =  (20002, 300)
    English Embeddings shape = (20002, 300)

    Hindi_vectors_shape = (122864, 78)
    English_vectors_shape = (122864, 77)
    


In [8]:
eng_wts = np.load(datasets_path + '/English_Vectorizer_weights.npy', allow_pickle = True)
hin_wts = np.load(datasets_path + '/Hindi_Vectorizer_weights.npy', allow_pickle = True)

In [23]:
english_vectorizer = tf.keras.layers.TextVectorization()
english_vectorizer.set_vocabulary(eng_wts[0])
hindi_vectorizer = tf.keras.layers.TextVectorization()
hindi_vectorizer.set_vocabulary(hin_wts[0])

In [10]:
print(english_vectorizer.get_vocabulary()[:20])
print(hindi_vectorizer.get_vocabulary()[:20])

['', '[UNK]', '<pad>', '<sos>', '<eos>', 'the', 'of', 'and', 'to', 'in', 'a', 'is', 'that', 'was', 'it', 'for', 'this', 'are', 'on', 'as']
['', '[UNK]', '<pad>', '<sos>', '<eos>', 'के', 'में', '.', 'की', 'है', 'और', 'से', 'का', ',', 'को', 'हैं', 'एक', 'कि', 'पर', 'भी']


##### Note-
    -> Embeddings Dimensions for both Hindi and English texts = 300
    -> The Text vectors have a vocab size of 20_000, as we limited it to 20_000 in the keras TextVectorization layer during data preprocessing.
    -> The sentence vectors, for hindi has 78 dimensions, while for English there are 77 dimensions.
    -> There are 122_864 samples in parallel for both the features(English) and targets(Hindi)

In [26]:
BATCH_SIZE = 64
MAX_SEQ_LEN = 77
EMBEDDING_DIMS = 300
VOCAB_SIZE = Hindi_embedding_matrix.shape[0]                    # Same for both the Hindi and English texts

In [29]:
def format_dataset(english, hindi):
    return (english, hindi[:, :-1], hindi[:, 1:])



dataset = tf.data.Dataset.from_tensor_slices((English_vectorized_text, Hindi_vectorized_text))
dataset = dataset.shuffle(200_000)
train = dataset.take(100_000)
test = dataset.skip(100_000)
train = (train.batch(BATCH_SIZE).map(format_dataset)).prefetch(tf.data.AUTOTUNE).cache()
test = test.batch(BATCH_SIZE).map(format_dataset).prefetch(tf.data.AUTOTUNE).cache()

In [30]:
for enc_inputs, dec_inputs, targets in train.take(1):
    print(f'encoder_inputs.shape: {enc_inputs.shape}')
    print(f'decoder_inputs.shape: {dec_inputs.shape}')
    print(f"targets.shape: {targets.shape}")

encoder_inputs.shape: (64, 77)
decoder_inputs.shape: (64, 77)
targets.shape: (64, 77)


## <center> Masking </center>
<p aligh = "justify" >Masking is a very simple but important factor in improving the accuracy of a model. It helps to tell a model, which parts of the input data to consider, at a particular Timestep. </p>

<br>

### <center> Types Of Masks </center>

####  Padding Mask
<p align = "justify">
We have already padded our sequences towards the end, to keep the sequence size consistent over the training examples. But, if this data is fed directly to the model, it might get something from the padded portions, which don't even exist in the sequence. So, we send along with our inputs, a padding mask function, that masks the padded region.
</p>

####  Look Ahead Mask
<p align = "justify">
The Transformer Network has an Encoder-Decoder Architecture. The decoder architecture tries to predict the text for next timestep, only considering the past timesteps. Look Ahead Mask allows us to mask the upcoming sections of the text.
</p>


In [31]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    ## Adding required dimensions to the padding
    return seq[:, tf.newaxis, tf.newaxis, :]        # (Batch_size, 1, 1, seq_len)


# Working Example
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(x)

<tf.Tensor: shape=(3, 1, 1, 5), dtype=float32, numpy=
array([[[[0., 0., 1., 1., 0.]]],


       [[[0., 0., 0., 1., 1.]]],


       [[[1., 1., 1., 0., 0.]]]], dtype=float32)>

In [32]:
def create_lookahead_mask(size):
    n = int(size * (size+1) / 2)
    mask = tfp.math.fill_triangular(tf.ones((n,), dtype=tf.int32), upper=False)
    return tf.cast(mask, tf.float32)

# Working Example
x = tf.random.uniform((1, 3))
temp = create_lookahead_mask(x.shape[1])
print(x)
print(temp)

tf.Tensor([[0.5741197  0.14189708 0.16985106]], shape=(1, 3), dtype=float32)
tf.Tensor(
[[1. 0. 0.]
 [1. 1. 0.]
 [1. 1. 1.]], shape=(3, 3), dtype=float32)


## <center> Positional Encoding </center>
<p align = "justify">
Attention layers see their input as a set of vectors, with no sequential order. This model also doesn't contain any recurrent or convolutional layers. Because of this a "positional encoding" is added to give the model some information about the relative position of the tokens in the sentence.<br>
The positional encoding vector is added to the embedding vector. Embeddings represent a token in a d-dimensional space where tokens with similar meaning will be closer to each other. But the embeddings do not encode the relative position of tokens in a sentence. So after adding the positional encoding, tokens will be closer to each other based on the similarity of their meaning and their position in the sentence, in the d-dimensional space.

$$\Large{PE_{(pos, 2i)} = \sin(pos / 10000^{2i / d_{model}})} $$
$$\Large{PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i / d_{model}})} $$
<center>
<img src = "https://camo.githubusercontent.com/c279dcae2225189217ab2827711b56b10919d179aead5584e87426de6ee27a67/68747470733a2f2f6a696e676c6573636f64652e6769746875622e696f2f6173736574732f696d672f706f7374732f696c6c75737472617465642d67756964652d7472616e73666f726d65722d31302e6a7067" width = "800"> </center>

In [33]:
def get_positional_encoding(num_positions:int, dimensions:int):
    """
    num_positions: Length Of Sequences in the dataset after padding
    dimensions: Number of dimensions used to represent each word in embedding matrix
    """
    # Create a column vector for positions
    pos_vec = np.arange(num_positions)[:, np.newaxis]
    
    # Create a row vector for dimensions
    dims_vec = np.arange(dimensions)[np.newaxis, :]
    
    i = dims_vec // 2
    angles = pos_vec * 1.0 / (np.power(10_000, 2 * i / np.float32(dimensions)))
    angles[:, 0::2] = np.sin(angles[:, 0::2])
    angles[:, 0::1] = np.cos(angles[:, 0::1])
    pos_encoding = angles[np.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

get_positional_encoding(10, 4)

<tf.Tensor: shape=(1, 10, 4), dtype=float32, numpy=
array([[[ 1.        ,  1.        ,  1.        ,  1.        ],
        [ 0.66636676,  0.5403023 ,  0.99995   ,  0.99995   ],
        [ 0.6143003 , -0.41614684,  0.9998    ,  0.9998    ],
        [ 0.9900591 , -0.9899925 ,  0.99955016,  0.99955004],
        [ 0.7270351 , -0.6536436 ,  0.9992005 ,  0.9992001 ],
        [ 0.5744009 ,  0.2836622 ,  0.9987513 ,  0.99875027],
        [ 0.9612168 ,  0.96017027,  0.9982027 ,  0.99820054],
        [ 0.7918362 ,  0.75390226,  0.997555  ,  0.997551  ],
        [ 0.5492263 , -0.14550003,  0.9968085 ,  0.99680173],
        [ 0.9162743 , -0.91113025,  0.99596363,  0.9959527 ]]],
      dtype=float32)>

# <center> Transformer </center>
<center>
<img src = "https://www.tensorflow.org/images/tutorials/transformer/transformer.png" height = 400></img>

#### Encoder
<p align = "justify">
The Transformer Encoder layer pairs self-attention and convolutional neural network style of processing to improve the speed of training and passes K and V matrices to the Decoder. The inputs are passed through multiple layers of similar structure, to get the outputs to be fed to Decoder. The dimensions of the inputs are secured as they are, because blocks are repeated.
</p>
###### Steps-
    -> Input Sequence is

<img src = "https://jalammar.github.io/images/t/transformer_resideual_layer_norm.png" height = "300"> </img>

In [34]:
class Encoder_Layer(Layer):
    def __init__(self, embedding_dims, num_heads, fully_connected_dim,
                 dropout_rate = 0.1, layernorm_eps = 1e-6):
        super(Encoder_Layer, self).__init__()

        self.mha = MultiHeadAttention(num_heads,
                                      key_dim = embedding_dims,
                                      dropout = dropout_rate)
        self.dense1 = Dense(fully_connected_dim, activation = 'relu')
        self.dense2 = Dense(embedding_dims)
        
        self.layer_norm1 = LayerNormalization(epsilon = layernorm_eps)
        self.layer_norm2 = LayerNormalization(epsilon = layernorm_eps)

        self.dropout_ffn = Dropout(dropout_rate)

    def call(self, x, mask):
        attnout = self.mha(x, x, x, mask)
        attnout = self.layer_norm1(x + attnout)

        ffn_out = self.dense1(attnout)
        ffn_out = self.dense2(attnout)
        ffn_out = self.dropout_ffn(ffn_out)
        encoder_layer_out = self.layer_norm2(ffn_out + attnout)
        return encoder_layer_out

In [35]:
class Encoder(Layer):
    def __init__(self, num_layers, num_heads,
                 embedding_dims, fully_connected_dim,
                 input_vocab_size, sequence_len,
                 dropout_rate = 0.1, layernorm_eps = 1e-6):
        super(Encoder, self).__init__()
        
        self.embedding_dims = embedding_dims
        self.num_layers = num_layers
        self.sequence_len = sequence_len
        self.embedding = Embedding(input_vocab_size, embedding_dims,
                                embeddings_initializer = tf.keras.initializers.Constant(English_embedding_matrix))
        
        self.positional_encoding = get_positional_encoding(self.sequence_len, embedding_dims)

        self.encoding_layers = [Encoder_Layer(
                                    embedding_dims = embedding_dims,
                                    num_heads = num_heads,                    
                                    fully_connected_dim = fully_connected_dim,                                
                                    dropout_rate = dropout_rate, layernorm_eps = layernorm_eps
                                ) for i in range(self.num_layers)]
        self.dropout = Dropout(dropout_rate)


    def call(self, x, padding_mask = None):

        x = self.embedding(x)

        ## Scaling the embeddings
        x *= tf.math.sqrt(tf.cast(self.embedding_dims, tf.float32))

        ## Adding the positional encodings to embeddings
        x += self.positional_encoding[:, :self.sequence_len, :]
        
        x = self.dropout(x)

        # Passing x through series of encoder_layers:
        for i in range(self.num_layers):
            x = self.encoding_layers[i](x, mask = padding_mask)
        # x = self.encoding_layers[0](x, mask = padding_mask)
        return x



In [36]:
sample_encoder = Encoder(num_layers = 2, num_heads = 4,
                 embedding_dims = EMBEDDING_DIMS, fully_connected_dim = 512,
                 input_vocab_size = VOCAB_SIZE, sequence_len = MAX_SEQ_LEN)
temp_input = tf.random.uniform((128, 77), dtype=tf.int64, minval=0, maxval=200)
temp_padding_mask = create_padding_mask(temp_input)
sample_encoder_output = sample_encoder(temp_input, temp_padding_mask)

print(sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

(128, 77, 300)


#### Decoder
<p align = "justify">
Similar to the Encoder, Decoder has multiple decoder_layers.
</p>

<img src = "https://www.mihaileric.com/static/output_token-06e143fae69ea58572608d65a546255c-06aef.png" height = 400></img>

In [37]:
class Decoder_Layer(Layer):
    def __init__(self, num_heads, embedding_dims, fully_connected_dim, dropout_rate = 0.1, layernorm_eps = 1e-6):
        super(Decoder_Layer, self).__init__()

        self.mha1 = MultiHeadAttention(num_heads, embedding_dims, dropout = dropout_rate)
        self.layernorm1 = LayerNormalization(epsilon = layernorm_eps)

        self.mha2 = MultiHeadAttention(num_heads, embedding_dims, dropout = dropout_rate)
        self.layernorm2 = LayerNormalization(epsilon = layernorm_eps)

        self.dense1 = Dense(fully_connected_dim, activation = 'relu')
        self.dense2 = Dense(embedding_dims)
        self.dropout_ = Dropout(dropout_rate)
        self.layernorm3 = LayerNormalization(epsilon = layernorm_eps)

    def call(self, x, encoder_out, padding_mask, lookahead_mask):
        attn1_out = self.mha1(x, x, x, lookahead_mask)
        attn1_out = self.layernorm1(x + attn1_out)

        attn2_out = self.mha2(attn1_out, encoder_out, encoder_out, padding_mask)
        attn2_out = self.layernorm2(attn1_out + attn2_out)

        dense_out = self.dense1(attn2_out)
        dense_out = self.dense2(dense_out)
        dense_out = self.dropout_(dense_out)

        decoder_out = self.layernorm3(dense_out + attn2_out)
        return decoder_out

In [38]:
class Decoder(Layer):
    def __init__(self, num_layers, num_heads,
                 embedding_dims, fully_connected_dim,
                 output_vocab_size, sequence_length,
                 dropout_rate = 0.1,
                 layernorm_eps = 1e-6
                 ):
        super(Decoder, self).__init__()
        self.num_layers = num_layers
        self.seq_len = sequence_length
        self.embedding_dims = embedding_dims

        self.embedding = Embedding(output_vocab_size, embedding_dims,
                            embeddings_initializer = tf.keras.initializers.Constant(Hindi_embedding_matrix))
        self.positional_encoding = get_positional_encoding(sequence_length, embedding_dims)

        self.decoder_layers = [Decoder_Layer(num_heads, embedding_dims,
                                              fully_connected_dim, dropout_rate,
                                              layernorm_eps) for i in range(num_layers)]


        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)
        self.output_layer = Dense(VOCAB_SIZE, activation='softmax')

    def call(self, x, encoder_out, padding_mask, lookahead_mask):
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.embedding_dims, tf.float32))
        x += self.positional_encoding[:, :self.seq_len, :]
        x = self.dropout1(x)

        for i in range(self.num_layers):
            x = self.decoder_layers[i]( x, encoder_out, padding_mask, lookahead_mask)
        x = self.dropout2(x)
        x = self.output_layer(x)

        return x

In [39]:
temp_lookahead_mask = create_lookahead_mask(77)
print(temp_lookahead_mask.shape)
sample_decoder = Decoder(num_layers = 2, num_heads = 4,
                 embedding_dims = EMBEDDING_DIMS, fully_connected_dim = 512,
                 output_vocab_size = VOCAB_SIZE, sequence_length = MAX_SEQ_LEN)


output = sample_decoder(temp_input, sample_encoder_output, temp_padding_mask, temp_lookahead_mask)
output.shape

(77, 77)


TensorShape([128, 77, 20002])

### Assembling The Model

In [40]:
class Transformer_Model(tf.keras.Model):

    def __init__(self, sequence_length,
                 num_encoder_layers = 2, num_encoder_heads = 4,
                 input_embedding_dims = EMBEDDING_DIMS, fully_connected_dim = EMBEDDING_DIMS,
                 input_vocab_size = VOCAB_SIZE, num_decoder_layers = 2, num_decoder_heads = 4,
                 output_embedding_dims = EMBEDDING_DIMS, output_vocab_size = VOCAB_SIZE
                 ):
        super(Transformer_Model, self).__init__()
        self.lookahead_mask = create_lookahead_mask(sequence_length)

        self.encoder_ = Encoder(num_encoder_layers, num_encoder_heads, input_embedding_dims,
                        fully_connected_dim, input_vocab_size, sequence_length,
                        dropout_rate = 0.3)
        

        self.decoder_ = Decoder(num_decoder_layers, num_decoder_heads,
                    output_embedding_dims, fully_connected_dim,
                    output_vocab_size, sequence_length,
                    dropout_rate = 0.3)
            

    def call(self, encoder_inputs, decoder_inputs):
        encoder_padding_mask = create_padding_mask(encoder_inputs)
        decoder_padding_mask = create_padding_mask(decoder_inputs)
        encoder_out = self.encoder_(encoder_inputs, encoder_padding_mask)
        decoder_out = self.decoder_(decoder_inputs, encoder_out, decoder_padding_mask, self.lookahead_mask)
        return decoder_out

In [41]:
sample_transformer = Transformer_Model(77)

In [42]:
temp_enc_input = tf.random.uniform((64, 77), dtype=tf.int64, minval=0, maxval=200)
temp_dec_input = tf.random.uniform((64, 77), dtype=tf.int64, minval=0, maxval=200)

output = sample_transformer(temp_enc_input, temp_dec_input)
output.shape

TensorShape([64, 77, 20002])

In [43]:
sample_transformer.summary()
del(sample_transformer)

Model: "transformer__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  9252000   
                                                                 
 decoder_1 (Decoder)         multiple                  18161602  
                                                                 
Total params: 27,413,602
Trainable params: 27,413,602
Non-trainable params: 0
_________________________________________________________________


### Model Training

In [44]:
with strategy.scope():
    transformer = Transformer_Model(MAX_SEQ_LEN)

    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        reduction = tf.keras.losses.Reduction.NONE
    )

    def compute_loss(labels, predictions):
        per_example_loss = loss_object(labels, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync)


    test_loss = tf.keras.metrics.Mean(name = 'test_loss')

    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

    lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate = 0.001,
        decay_steps = 50,
        decay_rate = 0.9
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

    @tf.function                                                                # Converts the function in graph mode for faster execution
    def distributed_training_step(datasets_inputs):
        per_replica_losses = strategy.run(train_steps, args = (datasets_inputs, ))
        print(per_replica_losses)
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis = None)

    @tf.function
    def distributed_test_step(datasets_inputs):
        strategy.run(test_steps, args = (datasets_inputs, ))

    def train_steps(inputs):
        encoder_inputs, decoder_inputs, targets = inputs

        # Shape of images here -> [128, 224, 224, 3]  with 128 being batch size.
        # Shape of Labels here -> (128, 9) representing 9 classes.

        with tf.GradientTape() as tape:                                         # To keep in check gradients of weights and biases(kernel)
            predictions = transformer(encoder_inputs, decoder_inputs)
            loss = compute_loss(targets, predictions)

        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))    # Applying changes in weights and biases.

        train_accuracy.update_state(targets, predictions)

        return loss

    def test_steps(inputs):
        encoder_inputs, decoder_inputs, targets = inputs

        predictions = transformer(encoder_inputs, decoder_inputs)
        loss = loss_object(targets, predictions)

        test_loss.update_state(loss)
        test_accuracy.update_state(targets, predictions)

In [ ]:
NUM_EPOCHS = 10
with strategy.scope():
    for epoch in range(NUM_EPOCHS):
        epoch_start = datetime.now()
        total_loss = 0.0
        num_batches = 0

        # Training Loop
        for x in train: 
            total_loss += distributed_training_step(x)
            num_batches += 1

        train_loss = total_loss / num_batches

        # Testing Loop
        for x in test:
            distributed_test_step(x)

        epoch_end = datetime.now()
        
        template = ("Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}, Test Loss: {:.2f}, Test Accuracy: {:.2f}, \t Elapsed Time: {}")

        print(template.format(
            epoch + 1,
            train_loss,
            train_accuracy.result() * 100,
            test_loss.result() / strategy.num_replicas_in_sync,
            test_accuracy.result() * 100,
            (epoch_end - epoch_start).seconds
        ))

        test_loss.reset_states()
        train_accuracy.reset_states()

        test_accuracy.reset_states()

PerReplica:{
  0: Tensor("output_0_shard_0:0", shape=(), dtype=float32),
  1: Tensor("output_0_shard_1:0", shape=(), dtype=float32),
  2: Tensor("output_0_shard_2:0", shape=(), dtype=float32),
  3: Tensor("output_0_shard_3:0", shape=(), dtype=float32),
  4: Tensor("output_0_shard_4:0", shape=(), dtype=float32),
  5: Tensor("output_0_shard_5:0", shape=(), dtype=float32),
  6: Tensor("output_0_shard_6:0", shape=(), dtype=float32),
  7: Tensor("output_0_shard_7:0", shape=(), dtype=float32)
}


PerReplica:{
  0: Tensor("output_0_shard_0:0", shape=(), dtype=float32),
  1: Tensor("output_0_shard_1:0", shape=(), dtype=float32),
  2: Tensor("output_0_shard_2:0", shape=(), dtype=float32),
  3: Tensor("output_0_shard_3:0", shape=(), dtype=float32),
  4: Tensor("output_0_shard_4:0", shape=(), dtype=float32),
  5: Tensor("output_0_shard_5:0", shape=(), dtype=float32),
  6: Tensor("output_0_shard_6:0", shape=(), dtype=float32),
  7: Tensor("output_0_shard_7:0", shape=(), dtype=float32)
}


PerReplica:{
  0: Tensor("output_0_shard_0:0", shape=(), dtype=float32),
  1: Tensor("output_0_shard_1:0", shape=(), dtype=float32),
  2: Tensor("output_0_shard_2:0", shape=(), dtype=float32),
  3: Tensor("output_0_shard_3:0", shape=(), dtype=float32),
  4: Tensor("output_0_shard_4:0", shape=(), dtype=float32),
  5: Tensor("output_0_shard_5:0", shape=(), dtype=float32),
  6: Tensor("output_0_shard_6:0", shape=(), dtype=float32),
  7: Tensor("output_0_shard_7:0", shape=(), dtype=float32)
}
Epoch 1, Loss: 155.57, Accuracy: 78.19, Test Loss: 0.19, Test Accuracy: 79.08, 	 Elapsed Time: 215
Epoch 2, Loss: 114.19, Accuracy: 79.65, Test Loss: 0.19, Test Accuracy: 79.56, 	 Elapsed Time: 132
Epoch 3, Loss: 113.27, Accuracy: 79.73, Test Loss: 0.18, Test Accuracy: 79.63, 	 Elapsed Time: 132
Epoch 4, Loss: 113.18, Accuracy: 79.76, Test Loss: 0.18, Test Accuracy: 79.63, 	 Elapsed Time: 132
Epoch 5, Loss: 113.18, Accuracy: 79.76, Test Loss: 0.18, Test Accuracy: 79.63, 	 Elapsed Time: 133
Epoch 6, Lo

In [ ]:
### Runtime disconnected while running😑😑